<a href="https://colab.research.google.com/github/hansong0219/Advanced-DeepLearning-Study/blob/master/improved_GAN/WGAN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# WGAN


In [2]:
from tensorflow.keras.layers import Activation, Dense, Input
from tensorflow.keras.layers import Conv2D, Flatten
from tensorflow.keras.layers import Reshape, Conv2DTranspose
from tensorflow.keras.layers import LeakyReLU
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.layers import concatenate
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import RMSprop

from tensorflow.keras.datasets import mnist
from tensorflow.keras.models import load_model
from tensorflow.keras import backend as K


import numpy as np
import math
import matplotlib.pyplot as plt

# 생성기와 판별기 등 함수 구성 
생성기와 판별기의 함수는 DCGAN의 구성을 그대로 사용한다.

In [13]:
def build_generator(inputs,
              image_size,
              activation='sigmoid'):

    image_resize = image_size // 4
    kernel_size = 5
    layer_filters = [128, 64, 32, 1]
    
    x = inputs
    x = Dense(image_resize * image_resize * layer_filters[0])(x)
    x = Reshape((image_resize, image_resize, layer_filters[0]))(x)

    for filters in layer_filters:
        if filters > layer_filters[-2]:
            strides = 2
        else:
            strides = 1
        x = BatchNormalization()(x)
        x = Activation('relu')(x)
        x = Conv2DTranspose(filters=filters,
                            kernel_size=kernel_size,
                            strides=strides,
                            padding='same')(x)

    if activation is not None:
        x = Activation(activation)(x)

    return Model(inputs, x, name='generator')


def build_discriminator(inputs,
                  activation='sigmoid'):

    kernel_size = 5
    layer_filters = [32, 64, 128, 256]

    x = inputs
    for filters in layer_filters:
        # first 3 convolution layers use strides = 2
        # last one uses strides = 1
        if filters == layer_filters[-1]:
            strides = 1
        else:
            strides = 2
        x = LeakyReLU(alpha=0.2)(x)
        x = Conv2D(filters=filters,
                   kernel_size=kernel_size,
                   strides=strides,
                   padding='same')(x)

    x = Flatten()(x)
    # default output is probability that the image is real
    outputs = Dense(1)(x)
    if activation is not None:
        print(activation)
        outputs = Activation(activation)(outputs)

    return Model(inputs, outputs, name='discriminator')


def plot_images(generator,
                noise_input,
                noise_label=None,
                noise_codes=None,
                show=False,
                step=0,
                model_name="gan"):
  
    os.makedirs(model_name, exist_ok=True)
    filename = os.path.join(model_name, "%05d.png" % step)
    rows = int(math.sqrt(noise_input.shape[0]))
    if noise_label is not None:
        noise_input = [noise_input, noise_label]
        if noise_codes is not None:
            noise_input += noise_codes

    images = generator.predict(noise_input)
    plt.figure(figsize=(2.2, 2.2))
    num_images = images.shape[0]
    image_size = images.shape[1]
    for i in range(num_images):
        plt.subplot(rows, rows, i + 1)
        image = np.reshape(images[i], [image_size, image_size])
        plt.imshow(image, cmap='gray')
        plt.axis('off')
    plt.savefig(filename)
    if show:
        plt.show()
    else:
        plt.close('all')


def test_generator(generator):
    noise_input = np.random.uniform(-1.0, 1.0, size=[16, 100])
    plot_images(generator,
                noise_input=noise_input,
                show=True,
                model_name="test_outputs")

# WGAN 구현 

GAN 과 유사하게 생성기와 판별기를 교대로 훈련 시키지만 구현에 있어서 WGAN 의 가장 큰 특징은 Wasserstein Loss 를 사용함과 동시에 생성기를 1회 훈련시키기전에 판별기를 n 회 훈련 시킨다. 

이는 생성기와 판별기를 동일한 횟수로 훈련시키는 GAN 과는 다른 점이다. 판별기를 훈련시킨다는 것은 판별기의 매개변수를 학습한다는 것을 뜻한다. 

또, EMD의 제약 조건을 만족 시키기 위해 위해 립시츠 제약에 대한 변수 또한 필요하다. 립시츠 조건이란 두 점사이의 거리를 일정 비율 이상 증가시키지 않는다. 

In [8]:
def wasserstein_loss(y_label, y_pred):
  return - K.mean(y_label*y_pred)

In [19]:
#MNIST 데이터 세트 로딩
(x_train,_),(_,_) = mnist.load_data()

# 데이터 형상 변환 및 정규화
image_size = x_train.shape[1]
x_train = np.reshape(x_train, [-1, image_size, image_size, 1])
x_train = x_train.astype('float32')/255
  
model_name = "wgan_mnist"

#네트워크 매개 변수 지정 
latent_size = 100
#WGAN 에서 추가된 매개변수
n_critic = 5
clip_value = 0.01

batch_size = 64
lr = 5e-5
train_steps = 40000
input_shape = (image_size, image_size, 1)

In [10]:
#판별기 모델 구성
inputs = Input(shape=input_shape, name = 'discriminator_input')

#WGAN 은 선형 activation 을 사용한다. 
discriminator = build_discriminator(inputs, activation='linear')
optimizer = RMSprop(lr=lr)

#WGAN 판별기는 Wasserstein loss 를 사용한다.
discriminator.compile(loss=wasserstein_loss,optimizer=optimizer,metrics=["accuraty"])

discriminator.summary()

linear
Model: "discriminator"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
discriminator_input (InputLa [(None, 28, 28, 1)]       0         
_________________________________________________________________
leaky_re_lu_8 (LeakyReLU)    (None, 28, 28, 1)         0         
_________________________________________________________________
conv2d_8 (Conv2D)            (None, 14, 14, 32)        832       
_________________________________________________________________
leaky_re_lu_9 (LeakyReLU)    (None, 14, 14, 32)        0         
_________________________________________________________________
conv2d_9 (Conv2D)            (None, 7, 7, 64)          51264     
_________________________________________________________________
leaky_re_lu_10 (LeakyReLU)   (None, 7, 7, 64)          0         
_________________________________________________________________
conv2d_10 (Conv2D)           (None, 4, 4, 128)

In [14]:
#생성기 모델 구성
input_shape = (latent_size,)
inputs = Input(shape=input_shape, name = 'z_input')
generator = build_generator(inputs, image_size)
generator.summary()

Model: "generator"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
z_input (InputLayer)         [(None, 100)]             0         
_________________________________________________________________
dense_4 (Dense)              (None, 6272)              633472    
_________________________________________________________________
reshape (Reshape)            (None, 7, 7, 128)         0         
_________________________________________________________________
batch_normalization (BatchNo (None, 7, 7, 128)         512       
_________________________________________________________________
activation_3 (Activation)    (None, 7, 7, 128)         0         
_________________________________________________________________
conv2d_transpose (Conv2DTran (None, 14, 14, 128)       409728    
_________________________________________________________________
batch_normalization_1 (Batch (None, 14, 14, 128)       51

In [16]:
#적대적 모델 구성

#적대적 네트워크를 훈련하는 동안 판별기의 가중치는 고정
discriminator.trainable = False
adversarial = Model(inputs, discriminator(generator(inputs)),name = model_name)

adversarial.compile(loss=wasserstein_loss, optimizer=optimizer,metrics=['accuracy'])
adversarial.summary()

Model: "wgan_mnist"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
z_input (InputLayer)         [(None, 100)]             0         
_________________________________________________________________
generator (Functional)       (None, 28, 28, 1)         1301505   
_________________________________________________________________
discriminator (Functional)   (None, 1)                 1080577   
Total params: 2,382,082
Trainable params: 1,300,801
Non-trainable params: 1,081,281
_________________________________________________________________


In [20]:
models = (generator, discriminator, adversarial)
params = (batch_size, latent_size, n_critic, clip_value, train_steps, model_name)

# WGAN 훈련

In [21]:
def train(models, x_train, params):
  #판별기와 적대적 네트워크를 배치단위로 교대로 훈련 
  """
  먼저 판별기가 제대로 레이블이 붙은 진짜와 가짜이미지를 사용해 n_critic 번 훈련된다.
  판별기 가중치는 립시츠 조건에 따라 범위가 제한된다.
  다음으로 생성기가 가짜 이미지를 진짜인 것 처럼 적대적네트워크를 통해 훈련된다.
  """

  # 입력 인수 
  """
  models(list) : Generator, DIscriminator, Adversarial 모델
  x_train (tensor) : 이미지 훈련
  params (list) : 네트워크 매개변수
  """

  #GAN 모델
  generator, discriminator, adversarial = models

  #네트워크 매개변수
  (batch_size, latent_size, n_critic, clip_value, train_steps, model_name) = params

  #이미지 생성 단계
  save_interval = 500

  #훈련하는 동안의 생성기 출력을 확인하기 위한 노이즈 벡터
  noise_input = np.random.uniform(-1.0, 1.0, size=[16, latent_size])
  #훈련 데이터 세트의 수 
  train_size = x_train.sahpe[0]
  # 실제 데이터의 레이블
  real_labels = np.ones((batch_size, 1))

  for i in range (train_steps):
    loss = 0
    acc = 0
    
    #판별기를 n_critic 회 만큼 훈련을 우선 시킨다.
    for i in range(n_critic):
      # 배치별 판별기 훈련 
      # 실제 이미지와 label = 1, 가짜이미지 label = -1 로 구성된 1배치 
      # 데이터 셋에서  실제 이미지를 임의로 선정한다.
      rand_indices = np.random.randint(0, train_size, size=batch_size)
      real_images = x_train[rand_indices]
      
      # 생성기를 이용하여 가짜 이미지를 생성한다
      noise = np.random.uniform(-1.0, 1.0, size=[batch_size, latent_size])
      fake_images = generater.predict(noise)
      
      #판별기 네트워크의 훈련
      """
      - 진짜 데이터 레이블 = 1 , 가짜 데이터 레이블 = -1 로 구성한다.
      - 진짜와 가짜 이미지를 결합해 하나의 배치를 만드는 대신, 처음에는 진짜 데이터로 구성된 하나의 배치로 훈련한 다음 가짜 이미지로 구성된 하나의 배치로 훈련 하는 형태이다.
      (위와 같이 훈련함으로써 진짜와 가짜 데이터 레이블의 부호가 반대고 범위제한으로 인해 가중치의 크기가 작아서 경사가 소실되는 것을 방지한다.)
      """
      real_loss, real_acc = discriminator.train_on_batch(real_images, real_labels)
      fake_loss, fake_acc = discriminator.train_on_batch(fake_images,-real_labels)

      #평균 손실과 정확도를 누적 
      loss += 0.5*(real_loss+fake_loss)
      acc += 0.5*(real_acc+fake_acc)

      #립시츠 제약 사항을 만족하기 위한 판별치 가중 범위 제한
      for layer in discriminator.layers:
        weights = layer.get_weights()
        weights = [np.clip(weight, -clip_value, clip_value) for weight in weights]

    # n_critic 회 반복 훈련하는 동안 평균 손실과 정확도 
    loss /= n_critic
    acc /= n_critic
    log = "%d: [discriminator loss: %f, acc: %f]" %(i, loss, acc) 

    #1 배치 동안 적대적 네트워크의 훈련
    noise = np.random.uniform(-1.0, 1.0, size=[batch_size, latent_size])

    #적대적 네트워크 훈련 
    loss, acc = adversarial.train_on_batch(noise, real_labels)

    log = "%s [adversarial loss : %f, acc: %f]" %(log, loss, acc)
    print(log)
    
    if (i+1) % save_interval == 0:
      if (i+1) == train_steps:
        show = True
      else:
        show = False

      plot_images(generator, noise_input= noise_input, show=show, step=(i+1),model_name = model_name)

  generator.save(model_name + '.h5')